# Function dissection lab

1. What happens when we define a function?
2. Byte codes and Python compilation
3. Some attributes of the `__code__` attribute of a function object

In [1]:
def hello(name):
    return f'Hello, {name}!'

# What is the above?

I defined a function.

Let's be more specific:

1. 
